In [1]:
import os
import numpy as np
import requests
import pandas as pd
from rdkit import Chem
from rdkit import RDLogger
from rdkit.Chem import AllChem
from rdkit import DataStructs
from biopandas.pdb import PandasPdb

In [2]:
# Disable log messages from RDKit
RDLogger.DisableLog('rdApp.*')

In [3]:
PATH_TO_REFERENCE_LIGANDS_FOLDER = 'data/reference_ligands'
PATH_TO_PDB_FOLDER = 'data/pdb_files_originals'
PATH_TO_EDITED_PDB_FOLDER = 'data/pdb_files_edited'

In [4]:
inorganic_cofactors = ['MG', 'CU', 'MN', 'FE', 'MO', 'NI', 'ZN']

modified_residues = ['CME','ALY', 'OCS', 'CSO', 'NLE', '4AK', 'KCX']

atp_analogues = ['APC', 'ANP']

biologically_relevant = ['SAM', 'NAD', 'SAH', 'IMP', 'NDP', 'TTP', 'UMP','FPP'] + ['R1P', 'HPA', 'URA', 'PAU']

organic_cofactors_df = pd.read_csv('data/known_cofactors/known_cofactors.csv')

organic_cofactors_codes = organic_cofactors_df[~organic_cofactors_df['code'].isna()].code.tolist()

organic_cofactors_fps = [AllChem.GetMorganFingerprint(Chem.MolFromSmiles(smiles), 3)
                            for smiles in organic_cofactors_df.smiles]

buffer_components = pd.read_csv('data/crystal_buffer_components.csv').code.tolist()

bindingmoad_validity = pd.read_csv('data/tidy_BindingMOAD_2019.csv')[['Protein_ID', 'Ligand_Name','Ligand_Validity']].drop_duplicates()

In [8]:
referece_dictionary = create_reference_dictionary()

In [9]:
log = {'complex':[], 'ligands':[], 'failed':[], 'reason_for_failing':[]}

In [13]:
# 1H01_FAL, 5MHQ_8QT, 1OIR_HDY
for uniprot_id in referece_dictionary:
    os.makedirs(PATH_TO_EDITED_PDB_FOLDER+'/'+uniprot_id, exist_ok=True)
    for template in referece_dictionary[uniprot_id]:
        if template not in ['4HT2_V50.sdf']:
            continue
        print(template)
        #try:
        pdb_id = template.split('_')[0]
        template_id =template.split('_')[1].split('.')[0]

        # get pdb structure
        ppdb = PandasPdb().read_pdb(PATH_TO_PDB_FOLDER+'/'+uniprot_id+'/'+pdb_id+'.pdb')

        atom_df = ppdb.df['ATOM']
        hetatm_df = ppdb.df['HETATM']

        # get reference molecule
        reference = Chem.SDMolSupplier(PATH_TO_REFERENCE_LIGANDS_FOLDER+'/'+uniprot_id+'/'+template)[0]
        reference = reference.GetConformer()

        # calculate distances between reference ligand atoms to all other atoms in the structure
        i = 0
        series_hetatoms = {}
        for position in reference.GetPositions():
            series_hetatoms[i] = ppdb.distance(xyz=position, records=('HETATM'))
            i += 1
        # concatenate into a dataframe
        series_hetatoms = pd.concat(series_hetatoms, axis=1)
        # assign to each atom the minimum distance to the reference ligand
        series_hetatoms = series_hetatoms.min(axis=1)
        # add column with distances to the 'HETATM' biopandas dataframe
        hetatm_df = pd.concat([series_hetatoms, hetatm_df], axis=1)

        # create list of het residues that are less than 6.5 A from the reference ligand
        neibs = hetatm_df[hetatm_df[0]<6.5]['residue_name'].drop_duplicates().tolist()
        print(neibs)
        neibs = [neib for neib in neibs if neib not in ['HOH', template_id]+
                                                        inorganic_cofactors+
                                                        organic_cofactors_codes+
                                                        atp_analogues+
                                                        biologically_relevant+
                                                        modified_residues]

        to_remove = []
        # add them to the to_remove list if they are buffer components
        for neib in neibs:
            if neib in buffer_components:
                to_remove += [neib]
        neibs = [neib for neib in neibs if neib not in to_remove]

        # if there are still compounds check if they are similar to the cofactors
        probably_cofactor = []
        if len(neibs) > 0:
            for neib in neibs:
                print(neib)
                smiles = get_smiles_from_PDB_chem_comp(neib)
                if smiles != None:
                    smiles = smiles['SMILES_CANONICAL']
                    mol = Chem.MolFromSmiles(smiles)
                    if mol != None:
                        mol_neib_fp = AllChem.GetMorganFingerprint(mol, 3)
                        for fp in organic_cofactors_fps:
                            ts = DataStructs.TanimotoSimilarity(fp, mol_neib_fp)
                            if ts > 0.9:
                                probably_cofactor += [neib]

        neibs = [neib for neib in neibs if neib not in probably_cofactor]

        if len(neibs) > 0:
            # check if the ligands were defined as "invalid" or "Part of Protein" by BindingMOAD
            bm_validities = bindingmoad_validity[bindingmoad_validity['Protein_ID']==pdb_id].dropna().values.tolist()
            print(bm_validities)
            for neib in neibs:
                for row in bm_validities:
                    if neib in row[1]:
                        if row[2] == 'invalid':
                            to_remove += [neib]
                            break
                        if row[2] == 'Part of Protein':
                            probably_cofactor += [neib]
                            break

        neibs = [neib for neib in neibs if neib not in probably_cofactor+to_remove]

        if len(neibs) > 0:
            log['complex'] += [template.split('.')[0]]
            log['ligands'] += [','.join(neibs)]
        else:
            log['complex'] += [template.split('.')[0]]
            log['ligands'] += ['']

        to_remove = to_remove + neibs + [template_id]

        # Now remove from .pdb the het components in the to_remove list
        ppdb.df['HETATM'] = ppdb.df['HETATM'].loc[~ppdb.df['HETATM']['residue_name'].isin(to_remove)]
        print(to_remove)

        # save back to .pdb file
        ppdb.to_pdb(PATH_TO_EDITED_PDB_FOLDER+'/'+uniprot_id+'/'+template.split('.')[0]+'.pdb')

        log['failed'] += [0]
        log['reason_for_failing'] += ['']
        #except Exception as e:
        #    log['complex'] += [template.split('.')[0]]
        #    log['ligands'] += ['']
        #    log['failed'] += [1]
        #    log['reason_for_failing'] += [str(e)]
    #print('Complexes under uniprot_id',uniprot_id,'done!')
    #print('')

    #log_df = pd.DataFrame(log)
    #log_df.to_csv('data/logs/remove_hets_log.txt', index=False)

4HT2_V50.sdf
['ZN', 'V50', 'EDO', 'HOH']
['EDO', 'V50']


In [42]:
log_df = pd.read_csv('data/logs/remove_hets_log.txt')

In [65]:
log_df = pd.DataFrame(log)
log_df

,complex,ligands,failed,reason_for_failing
0,4WZV_E40,,0,
1,4WZV_E40,,0,
2,4WZV_E40,,0,
3,3MHO_J43,,0,
4,1D6W_00R,NA,0,
5,3KQE_LGM,NA,0,
6,3KQB_LGJ,NA,0,
7,3KQC_LGK,NA,0,
8,5ETM_5RW,,0,
9,5ETP_5RZ,,0,


In [39]:
undefined = np.array([[0,0,0]])
for lig in log_df[~log_df['ligands'].isna()]['ligands'].drop_duplicates().values:
    for i in lig.split(','):
        if i not in undefined[:,0]:
            smiles = get_smiles_from_PDB_chem_comp(i)
            if smiles != None:
                smiles = smiles['SMILES_CANONICAL']
                undefined = np.vstack([undefined,[i ,smiles, int(log_df.loc[(log_df['ligands']==lig)].shape[0])]])
            else:
                undefined = np.vstack([undefined,[i ,None, int(log_df.loc[(log_df['ligands']==lig)].shape[0])]])

/home/fol007/anaconda3/envs/py37_rdkit/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  after removing the cwd from sys.path.


In [40]:
undefined = undefined.tolist()[1:]

In [41]:
undefined.sort(key=lambda x:int(x[2]), reverse=True)
undefined

[['HG', '[Hg+2]', '2'],
 ['KCX', 'C(CCNC(=O)O)C[C@@H](C(=O)O)N', '2'],
 ['0MZ',
  'c1ccc(c(c1)/C=N/N=C\\2/N=C([C@@H](S2)CC(=O)Nc3cccc(c3)C(F)(F)F)O)O',
  '1'],
 ['MTX',
  'CN(Cc1cnc2c(n1)c(nc(n2)N)N)c3ccc(cc3)C(=O)N[C@@H](CCC(=O)O)C(=O)O',
  '1'],
 ['MG5',
  'CCC(=O)O[C@@H]1[C@H](O[C@H]([C@@H]([C@H]1OC(=O)CC)OC(=O)CC)O)COS(=O)(=O)N',
  '1'],
 ['6KE',
  'C[C@@H](C(=O)Nc1ccc(cc1)S(=O)(=O)N)SC2=NC(=C(C(=O)N2)C#N)c3ccc(cc3)Cl',
  '1'],
 ['1CR',
  'COc1cc2c3cc1OCCOc4cc5c(cc4OCc6cn(nn6)CCc7ccc(cc7)S(=O)(=O)N)Cc8cc9c(cc8Cc1cc(c(cc1C5)OCc1cn(nn1)CCC(=O)O)OCCOc1cc(c(cc1OC)Cc1cc(c(cc1C3)OC)OCCO9)C2)OCc1cn(nn1)CCC(=O)O',
  '1'],
 ['MTL', 'C([C@H]([C@H]([C@@H]([C@@H](CO)O)O)O)O)O', '1'],
 ['ARG', 'C(C[C@@H](C(=O)O)N)CNC(=[NH2+])N', '1'],
 ['BY7', 'Cc1cc(nc(c1)N)CCc2cc(cc(c2)C(F)(F)F)CCCN(C)C', '1'],
 ['8FD', 'Cc1cc(nc2c1ccc(c2)COc3cc(cc(c3)C#N)C[C@H](C)NC)N', '1'],
 ['KMD', 'Cc1cc(nc(c1)N)CCc2cc(cc(c2)F)CC[C@@H]3CCCN3C', '1'],
 ['ZZ3', 'Cc1nc(nc(n1)SC)N', '1'],
 ['XQK', 'Cc1c(c([nH]n1)c2ccno2)C(=O

In [24]:
for und in undefined:
    mol = Chem.MolFromSmiles(und[1])
    if mol != None:
        smiles = Chem.MolToSmiles(mol)
        und[1] = smiles
    else:
        und[1] = None

[['HG', '[Hg+2]', '2'],
 ['KCX', 'C(CCNC(=O)O)C[C@@H](C(=O)O)N', '2'],
 ['0MZ',
  'c1ccc(c(c1)/C=N/N=C\\2/N=C([C@@H](S2)CC(=O)Nc3cccc(c3)C(F)(F)F)O)O',
  '1'],
 ['MTX',
  'CN(Cc1cnc2c(n1)c(nc(n2)N)N)c3ccc(cc3)C(=O)N[C@@H](CCC(=O)O)C(=O)O',
  '1'],
 ['MG5',
  'CCC(=O)O[C@@H]1[C@H](O[C@H]([C@@H]([C@H]1OC(=O)CC)OC(=O)CC)O)COS(=O)(=O)N',
  '1'],
 ['6KE',
  'C[C@@H](C(=O)Nc1ccc(cc1)S(=O)(=O)N)SC2=NC(=C(C(=O)N2)C#N)c3ccc(cc3)Cl',
  '1'],
 ['1CR',
  'COc1cc2c3cc1OCCOc4cc5c(cc4OCc6cn(nn6)CCc7ccc(cc7)S(=O)(=O)N)Cc8cc9c(cc8Cc1cc(c(cc1C5)OCc1cn(nn1)CCC(=O)O)OCCOc1cc(c(cc1OC)Cc1cc(c(cc1C3)OC)OCCO9)C2)OCc1cn(nn1)CCC(=O)O',
  '1'],
 ['MTL', 'C([C@H]([C@H]([C@@H]([C@@H](CO)O)O)O)O)O', '1'],
 ['ARG', 'C(C[C@@H](C(=O)O)N)CNC(=[NH2+])N', '1'],
 ['BY7', 'Cc1cc(nc(c1)N)CCc2cc(cc(c2)C(F)(F)F)CCCN(C)C', '1'],
 ['8FD', 'Cc1cc(nc2c1ccc(c2)COc3cc(cc(c3)C#N)C[C@H](C)NC)N', '1'],
 ['KMD', 'Cc1cc(nc(c1)N)CCc2cc(cc(c2)F)CC[C@@H]3CCCN3C', '1'],
 ['ZZ3', 'Cc1nc(nc(n1)SC)N', '1'],
 ['XQK', 'Cc1c(c([nH]n1)c2ccno2)C(=O

In [47]:
e=log_df[~log_df['reason_for_failing'].isna()].complex.tolist()
e = [i+'.sdf' for i in e]
e

['4WZV_E40.sdf',
 '3MHO_J43.sdf',
 '1D6W_00R.sdf',
 '3KQE_LGM.sdf',
 '3KQB_LGJ.sdf',
 '3KQC_LGK.sdf',
 '5ETM_5RW.sdf',
 '5ETP_5RZ.sdf',
 '3ENZ_HPA.sdf']

In [7]:
# helper functions:

def create_reference_dictionary(path_to_reference_ligands_folder=PATH_TO_REFERENCE_LIGANDS_FOLDER):
    '''organize dictionary based on the reference ligands'''
    reference_dictionary = {}
    uniprot_ids = os.listdir(path_to_reference_ligands_folder)
    for uniprot_id in uniprot_ids:
        reference_dictionary[uniprot_id] = []
        for references in os.listdir(path_to_reference_ligands_folder + '/' + uniprot_id):
            reference_dictionary[uniprot_id] += [references]
    return reference_dictionary

def get_smiles_from_PDB_chem_comp(chem_comp, program='OpenEye OEToolkits'):
    '''get Isomeric SMILES from the PDB'''
    variables=dict(comp_id = chem_comp)
    query = '''query($comp_id: String!) {
            chem_comp(comp_id: $comp_id) {
               pdbx_chem_comp_descriptor {
                         type
                         program
                         descriptor
                        }
                      }
               }'''
    response = requests.post('http://data.rcsb.org/graphql', json={'query': query, 'variables': variables})
    dictionary = {}
    if response.status_code == 200:
        response = response.json()['data']['chem_comp']
        if response is not None:
            response = response['pdbx_chem_comp_descriptor']
            if response == None:
                return None
            for content in response:
                if content['type'] == 'SMILES_CANONICAL' and content['program'] == program:
                    dictionary[content['type']] = content['descriptor']
            if dictionary=={}:
                return None
            return dictionary
        else:
            return None
    else:
        return None
